# Feature/Model Dimension Debugger
This notebook helps you find why runtime state vectors have length 15 while your trained model expects 18.

It checks:
- The **expected input dim** from your saved buy/sell PyTorch checkpoints
- The **actual feature dim** in your built dataset (`features.npy` / optional `features.csv`)
- Per-ticker feature dims (if you have per-ticker saved CSV)
- The **API-time state builder** output length (if you point it to the function)

**Edit the paths in the first cell** to match your repo.

In [1]:
# --- Paths: edit these ---
PROJECT_ROOT = r"/Users/jacopo/Documents/uni/FinalProject/cm_3070_FP/code"
DATA_DIR     = r"/Users/jacopo/Documents/uni/FinalProject/cm_3070_FP/code/data"  # where features.npy lives
BUY_CKPT     = r"/Users/jacopo/Documents/uni/FinalProject/cm_3070_FP/code/models/buy_agent.pt"
SELL_CKPT    = r"/Users/jacopo/Documents/uni/FinalProject/cm_3070_FP/code/models/sell_agent.pt"

# Optional: if you saved CSV during build_features (--save_csv)
FEATURES_CSV = None  # e.g. r"/.../data/features.csv"
META_CSV     = None  # e.g. r"/.../data/row_meta.csv"

import os, sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_DIR:", DATA_DIR)


PROJECT_ROOT: /Users/jacopo/Documents/uni/FinalProject/cm_3070_FP/code
DATA_DIR: /Users/jacopo/Documents/uni/FinalProject/cm_3070_FP/code/data


In [2]:
# --- Helper: infer expected input dim from a checkpoint/model ---
def infer_expected_in_features(model: nn.Module) -> int:
    # if model has .net = nn.Sequential(...)
    if hasattr(model, "net") and isinstance(model.net, nn.Sequential):
        for m in model.net:
            if isinstance(m, nn.Linear):
                return int(m.in_features)
    # else, find first Linear anywhere
    for m in model.modules():
        if isinstance(m, nn.Linear):
            return int(m.in_features)
    raise RuntimeError("Could not infer input features (no nn.Linear found).")

def load_checkpoint(path: str):
    ckpt = torch.load(path, map_location="cpu")
    return ckpt

def describe_checkpoint(path: str):
    ckpt = load_checkpoint(path)
    # Some projects save raw state_dict, others save dict with keys.
    if isinstance(ckpt, dict) and "model_state_dict" in ckpt:
        keys = list(ckpt.keys())
        print(f"Checkpoint {os.path.basename(path)} is a dict with keys:", keys)
        input_dim = ckpt.get("input_dim")
        feat_names = ckpt.get("feature_names")
        if input_dim is not None:
            print("  input_dim (stored):", input_dim)
        if feat_names is not None:
            print("  feature_names (stored) len:", len(feat_names))
            print("  first 10:", feat_names[:10])
        return ckpt
    else:
        print(f"Checkpoint {os.path.basename(path)} appears to be a raw object/state_dict.")
        return ckpt

describe_checkpoint(BUY_CKPT)
describe_checkpoint(SELL_CKPT)


Checkpoint buy_agent.pt appears to be a raw object/state_dict.
Checkpoint sell_agent.pt appears to be a raw object/state_dict.


{'cfg': {'gamma': 0.99,
  'lr': 0.001,
  'batch_size': 64,
  'buffer_size': 200000,
  'target_update_freq': 500,
  'epsilon_start': 1.0,
  'epsilon_end': 0.05,
  'epsilon_decay_steps': 48000,
  'state_dim': 18,
  'n_actions': 2},
 'state_dict': OrderedDict([('net.0.weight',
               tensor([[-0.0346, -0.1603, -0.1516,  ..., -0.2059, -0.7193, -0.1594],
                       [-0.0152, -0.1232,  0.1241,  ..., -0.4886,  0.1563,  0.2725],
                       [-0.0945,  0.2238,  0.0696,  ...,  0.2968, -0.2249, -0.1018],
                       ...,
                       [-0.0203, -0.1697,  0.3470,  ...,  0.4655, -0.3510,  0.0058],
                       [ 0.0804,  0.1480, -0.2966,  ...,  1.0250, -0.3420, -0.2935],
                       [-0.2918, -0.2472,  0.0630,  ..., -0.4796,  0.0973, -0.1121]])),
              ('net.0.bias',
               tensor([-0.2649,  0.1101, -0.1570, -0.0650, -0.1043,  0.2918, -0.0434,  0.0820,
                        0.1510, -0.2430, -0.6842, -0.3728,  

In [3]:
# # --- Load your model class to infer expected dim from the actual network ---
# # Adjust imports if your classes live elsewhere.
# # from agents.ddqn_agent import MLPQNetwork  # change if your class name differs

# def load_dqn_from_state_dict(ckpt_path: str, hidden=128):
#     ckpt = torch.load(ckpt_path, map_location="cpu")

#     # If you stored metadata, you can read input_dim from it.
#     input_dim = None
#     state_dict = None

#     if isinstance(ckpt, dict) and "model_state_dict" in ckpt:
#         state_dict = ckpt["model_state_dict"]
#         input_dim = ckpt.get("input_dim")
#     elif isinstance(ckpt, dict) and any(k.startswith("net.") for k in ckpt.keys()):
#         state_dict = ckpt
#     else:
#         # Try common key
#         if isinstance(ckpt, dict) and "state_dict" in ckpt:
#             state_dict = ckpt["state_dict"]
#         else:
#             raise RuntimeError("Unknown checkpoint format. Inspect the ckpt in previous cell.")

#     # Infer input dim from first layer weight if not provided
#     if input_dim is None:
#         # find first linear weight
#         for k, v in state_dict.items():
#             if k.endswith("weight") and v.ndim == 2:
#                 input_dim = int(v.shape[1])
#                 break

#     if input_dim is None:
#         raise RuntimeError("Could not infer input_dim from state_dict.")

#     # Your DQN ctor signature may differ. Update as needed.
#     model = MLPQNetwork(input_dim=input_dim, hidden_dim=hidden, output_dim=3)  # output_dim placeholder
#     model.load_state_dict(state_dict, strict=True)
#     model.eval()
#     return model, input_dim

# buy_model, buy_dim = load_dqn_from_state_dict(BUY_CKPT)
# sell_model, sell_dim = load_dqn_from_state_dict(SELL_CKPT)

# print("BUY expected input dim:", buy_dim)
# print("SELL expected input dim:", sell_dim)


In [4]:
import torch, os
ckpt = torch.load(BUY_CKPT, map_location="cpu")
print("type:", type(ckpt))
if isinstance(ckpt, dict):
    print("top-level keys:", list(ckpt.keys())[:50])
    # print sample from the first dict-like value
    for k,v in ckpt.items():
        if isinstance(v, dict):
            print("\nFirst nested dict key:", k, "nested keys sample:", list(v.keys())[:20])
            break
else:
    print("repr:", repr(ckpt)[:500])

type: <class 'dict'>
top-level keys: ['cfg', 'state_dict', 'target_state_dict', 'eps', 'steps', 'learn_steps']

First nested dict key: cfg nested keys sample: ['gamma', 'lr', 'batch_size', 'buffer_size', 'target_update_freq', 'epsilon_start', 'epsilon_end', 'epsilon_decay_steps', 'state_dim', 'n_actions']


In [5]:
import torch
import torch.nn as nn

def is_tensor_state_dict(d: dict) -> bool:
    if not isinstance(d, dict) or not d:
        return False
    # state_dict values are usually tensors or tensor-like
    return any(torch.is_tensor(v) for v in d.values())

def extract_state_dict(ckpt):
    # If it's already a tensor state_dict
    if isinstance(ckpt, dict) and is_tensor_state_dict(ckpt):
        return ckpt

    # Common wrappers
    if isinstance(ckpt, dict):
        for key in ["model_state_dict", "state_dict", "q_net_state_dict", "policy_state_dict"]:
            if key in ckpt and isinstance(ckpt[key], dict) and is_tensor_state_dict(ckpt[key]):
                return ckpt[key]

        # Search nested dicts one level deep
        for k, v in ckpt.items():
            if isinstance(v, dict) and is_tensor_state_dict(v):
                return v

        # Search deeper (recursive)
        stack = [ckpt]
        while stack:
            cur = stack.pop()
            if isinstance(cur, dict):
                if is_tensor_state_dict(cur):
                    return cur
                for v in cur.values():
                    if isinstance(v, dict):
                        stack.append(v)

    # If checkpoint is a Module saved directly
    if isinstance(ckpt, nn.Module):
        return ckpt.state_dict()

    raise RuntimeError("Could not find a tensor state_dict inside the checkpoint.")

In [7]:
import torch
import torch.nn as nn

class MLPQNetwork(nn.Module):
    def __init__(self, state_dim: int, n_actions: int, hidden_sizes=(128, 128)):
        super().__init__()
        layers = []
        last = state_dim
        for h in hidden_sizes:
            layers.append(nn.Linear(last, h))
            layers.append(nn.ReLU())
            last = h
        layers.append(nn.Linear(last, n_actions))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

def load_mlp_from_ckpt(path: str):
    ckpt = torch.load(path, map_location="cpu")
    cfg = ckpt["cfg"]
    sd  = ckpt["state_dict"]

    model = MLPQNetwork(state_dim=cfg["state_dim"], n_actions=cfg["n_actions"])
    model.load_state_dict(sd, strict=True)
    model.eval()
    return model, cfg

buy_model, buy_cfg = load_mlp_from_ckpt(BUY_CKPT)
sell_model, sell_cfg = load_mlp_from_ckpt(SELL_CKPT)

print("BUY cfg:", buy_cfg)
print("SELL cfg:", sell_cfg)
print("BUY in_features:", buy_model.net[0].in_features)

BUY cfg: {'gamma': 0.99, 'lr': 0.001, 'batch_size': 64, 'buffer_size': 200000, 'target_update_freq': 500, 'epsilon_start': 1.0, 'epsilon_end': 0.05, 'epsilon_decay_steps': 40000, 'state_dim': 15, 'n_actions': 2}
SELL cfg: {'gamma': 0.99, 'lr': 0.001, 'batch_size': 64, 'buffer_size': 200000, 'target_update_freq': 500, 'epsilon_start': 1.0, 'epsilon_end': 0.05, 'epsilon_decay_steps': 48000, 'state_dim': 18, 'n_actions': 2}
BUY in_features: 15


In [ ]:
# --- Inspect your built dataset dimensions ---
features_npy = os.path.join(DATA_DIR, "features.npy")
prices_npy   = os.path.join(DATA_DIR, "prices.npy")

X = np.load(features_npy)
y = np.load(prices_npy)

print("features.npy shape:", X.shape)  # (N, D)
print("prices.npy shape:", y.shape)

D = X.shape[1]
print("Built feature dim D =", D)


In [ ]:
# --- If you saved row_meta.csv / features.csv, check per-ticker consistency ---
if META_CSV and FEATURES_CSV and os.path.exists(META_CSV) and os.path.exists(FEATURES_CSV):
    meta = pd.read_csv(META_CSV)
    feats = pd.read_csv(FEATURES_CSV)
    print("meta:", meta.shape, "feats:", feats.shape)

    # join row index alignment
    assert len(meta) == len(feats)

    # check tickers present
    print("tickers:", meta['ticker'].unique())

    # feature column count
    print("feature cols:", feats.shape[1])

    # sanity: any NaNs?
    nan_rate = feats.isna().mean().sort_values(ascending=False).head(10)
    print("Top NaN rates:", nan_rate)
else:
    print("Skipping per-ticker checks. Set META_CSV and FEATURES_CSV paths if you have them.")


In [ ]:
# --- Key hypothesis: API-time state builder differs from training-time builder ---
# If you can import the function that builds a single 'state' for inference, call it here
# and print its length for a few tickers.
#
# Example (you must adapt to your project):
#
# from decision.state_builder import build_state_for_ticker
# for t in ["NVDA","AAPL","MSFT"]:
#     s = build_state_for_ticker(t, as_of=None)
#     print(t, len(s), s[:5])
#
# The goal is to find why build_state_for_ticker returns 15 values.
#
print("Next step: point this cell to the exact function used by DecisionEngine to build `state` at inference time.")


## What to look for
If `features.npy` has D=18 but API-time state has len=15, then your **inference feature builder** is not using the same schema.

Common causes:
- different config loaded by API (different `technical_indicators` list)
- sentiment disabled/enabled inconsistently
- state builder dropping missing cols/NaNs

If `features.npy` has D=15, but your checkpoint expects 18, you need to **retrain** or **load a checkpoint trained with the new schema**.